In [ ]:
import torch
from torch import nn, optim
import numpy as np

def create_model(num_features, sequence_len, k):
    network = nn.Sequential()
    
    # first convolutional layer, followed by relu and maxpooling layers
    network.add_module('Conv1', nn.Conv2d(in_channels=4, out_channels=16, kernel_size=(1, num_features)))
    network.add_module('Conv2', nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(k, 1)))
    network.add_module('ReLU1', nn.ReLU())
    network.add_module('MaxPool1', nn.MaxPool2d(kernel_size=(2,1), stride=2))
    
    # next layer, same thing.
    network.add_module('Conv3', nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(k, 1)))
    network.add_module('ReLU2', nn.ReLU())
    network.add_module('MaxPool2', nn.MaxPool2d(kernel_size=(2,1), stride=2))
    
    # then flatten everything into a single dimension and apply dropout
    network.add_module('Flatten', nn.Flatten())
    network.add_module('Dropout', nn.Dropout(0.2))
    
    # fully connected layers to produce a new set of features
    flat_size = 64 * ((sequence_len - k + 1) // 2 - k + 1) // 2
    network.add_module('FC1', nn.Linear(flat_size, 512))
    network.add_module('ReLU3', nn.ReLU())
    network.add_module('FC1', nn.Linear(512, 128))
    network.add_module('ReLU3', nn.ReLU())
    network.add_module('FC2', nn.Linear(128, num_features))

    return network

In [ ]:
def fetch_historical_observations(from_path, from_date, to_date):
    pass


In [36]:
historical_data = r"C:\Users\Austin\Desktop\Desktop Projects\flash\data\grand_island_nwt.csv"
normals_data = r"C:\Users\Austin\Desktop\Desktop Projects\flash\data\grand_island_normals.csv"

import pandas as pd

historical = pd.read_csv(historical_data)
normals = pd.read_csv(normals_data)

normals.insert(0, 'DAY', normals.index)


In [38]:
merged = pd.merge(historical, normals, on='DAY', how='left')

In [44]:
mean_columns = [x for x in merged.columns if 'mean' in x]
normals_for_historical = merged[['DATE', 'DAY'] + mean_columns]
normals_for_historical.columns = historical.columns
normals_for_historical

,DATE,DAY,AWND,PRCP,TMAX,TMIN,TAVG,WDF12,WSF12,UWDF12,VWDF12,DAY365,SIN,COS,TSPREAD
0,1984-01-01,1,12.023125,0.250000,36.437500,16.312500,26.375000,4.243332,21.081250,2.128157,-6.801978,0.005476,0.034398,0.999408,20.125000
1,1984-01-02,2,11.086875,1.687500,35.562500,13.625000,24.593750,4.603306,21.212500,9.214841,-8.775273,0.008214,0.051584,0.998669,21.937500
2,1984-01-03,3,11.770625,0.562500,32.062500,12.625000,22.343750,4.123340,21.393750,6.729583,-4.626903,0.010951,0.068755,0.997634,19.437500
3,1984-01-04,4,10.361875,3.312500,34.687500,16.375000,25.531250,4.068799,18.231250,0.852863,-2.693750,0.013689,0.085906,0.996303,18.312500
4,1984-01-05,5,10.695625,1.000000,29.937500,10.812500,20.375000,3.632467,19.081250,9.369700,-1.015564,0.016427,0.103031,0.994678,19.125000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14971,2024-12-27,362,12.106875,0.687500,36.250000,16.687500,26.468750,4.723297,21.568750,5.144142,-6.254100,0.993840,-0.038696,0.999251,19.562500
14972,2024-12-28,363,11.673125,1.250000,36.062500,16.000000,26.031250,4.385140,21.625000,5.241806,-7.215159,0.996578,-0.021501,0.999769,20.062500
14973,2024-12-29,364,10.290625,4.562500,36.312500,14.312500,25.312500,4.276057,19.400000,7.720220,-6.418250,0.999316,-0.004301,0.999991,22.000000
14974,2024-12-30,365,11.091865,7.125067,63.178218,39.722823,51.450521,3.704711,21.643082,1.256907,-2.574365,0.501395,0.000005,0.000050,23.455395


In [50]:
differences = historical.drop(['DATE', 'DAY'], axis=1).subtract(normals_for_historical.drop(['DATE', 'DAY'], axis=1))

In [52]:
squared_differences = differences.pow(2)
squared_differences

,AWND,PRCP,TMAX,TMIN,TAVG,WDF12,WSF12,UWDF12,VWDF12,DAY365,SIN,COS,TSPREAD
0,12.412410,7.562500,12.691406,93.847656,43.890625,0.414208,80.662852,0.000730,26.155003,0.000007,0.000296,1.969854e-07,37.515625
1,0.104410,2.847656,0.191406,28.890625,8.446289,0.583057,10.972656,525.649000,7.456315,0.000007,0.000295,5.469657e-07,24.378906
2,0.280238,0.316406,48.128906,178.890625,103.149414,0.011899,0.155039,409.176944,131.332304,0.000007,0.000295,1.071418e-06,41.441406
3,1.608141,10.972656,28.222656,276.390625,120.313477,1.800218,4.542227,90.174354,92.921216,0.000007,0.000294,1.769723e-06,127.972656
4,1.355769,1.000000,226.878906,634.410156,405.015625,5.297606,16.656602,22.332143,16.931071,0.000007,0.000293,2.641052e-06,102.515625
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14971,36.806972,0.472656,60.062500,127.972656,90.844727,0.000119,43.148477,26.462198,76.490766,0.000007,0.000295,6.617428e-07,12.691406
14972,10.068722,1.562500,481.253906,64.000000,224.063477,0.038553,0.075625,262.174567,138.081229,0.000007,0.000296,2.680975e-07,194.253906
14973,0.793213,20.816406,387.597656,278.472656,330.785156,1.713473,5.760000,28.116881,20.332337,0.000007,0.000296,4.925454e-08,9.000000
14974,2.862407,252.013505,448.516938,59.641998,208.817552,4.222515,3.038336,255.264480,54.399992,0.247925,0.000019,9.998815e-01,181.047663
